Lightcurve Plots for paper

In [ ]:
from common import * 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lightkurve import LightCurve

plt.rcParams.update({'font.size': 15, 'xtick.labelsize': 'small', 'ytick.labelsize': 'small','ytick.direction': 'in', 'xtick.direction': 'in',  # tells matplotlib to plot the ticks inward
                     'ytick.right': True, 'xtick.top': True, # tells matplotlib to plot the ticks also on the right and on the top
                     'xtick.minor.visible': True,'ytick.minor.visible': True, # include minor ticks as well
                     'xtick.major.width': 2, 'ytick.major.width': 2 # widht of major ticks
					 , 'ytick.minor.width': 1.25, 'xtick.minor.width': 1.25, 'axes.titlesize': 'small', 'axes.labelsize' : 'small'
                      })

In [ ]:
config_dict = get_config()
default_result_entry = 'all'

In [ ]:
def color(path):
    if 'BAb' in path or 'BLb' in path:
        return 'royalblue'
    elif 'BHr' in path or 'UBr' in path or 'BTr' in path:
        return 'firebrick'
    else:
        raise NameError('No Satellite Name in Path: ' + path)


        
def create_plot(path, fig, line_top):
    orbital_periods = {'UBr': 100.3708/1440, 'BAb':  100.3617/1440, 'BTr': 98.2428/1440, 'BLb': 99.6651/1440, 'BHr':97.0972/1440}
    start_times = [0, 603.4944567391507, 741.5047292323416, 776.5054984238852, 820.5036300755875, 904.4962054965513, 924.4950121321488, 1001.4959741645878, 1097.5042068469154, 1098.5042688566396, 1174.5043762953505, 1261.496833452413, 1321.4942997357778, 1370.4962364442279, 1373.4964575226386]
    end_times = [0, 733.5042846935763, 888.4974788652289, 805.5045879538453, 985.495045537583, 1071.5022059256232, 1097.5042068469154, 1169.5046546215021, 1175.5043165046843, 1263.4966751424365, 1340.4946352750571, 1407.499449583642, 1496.505482261617, 1442.5027029357807, 1418.5005081942502]
    name = path.split('/')[-1]
    data_path = os.path.join(config_dict['Decorrelation Path'],path)
    try:
        data = Data(path,None)
    except:
        return
    mosaic = """
            AABCD
            AABED
            """
#     fig = plt.figure(constrained_layout=True,figsize = (15,5),dpi = 100)
    axes = fig.subplot_mosaic(mosaic)
    lk = data._lk_obj
    time_for_ave =lk.time.value
    flux_for_ave = lk.flux.value
    sort = np.argsort(time_for_ave)
    time_for_ave = time_for_ave[sort]
    flux_for_ave = flux_for_ave[sort]
    orbit_per = orbital_periods[data.satellite]
    t_0  = 0
    for i in range(len(lk.time)):
        if time_for_ave[i + 1] - time_for_ave[i] > 0.5 * orbit_per:
            t0 = time_for_ave[i + 1] - 1.45 *orbit_per 
            break
    while t0 > time_for_ave[0]:
        t0 -= orbit_per
        
    t = t0
    ave_times = []
    ave_flux = []
#     print(ave_times)
    while t < time_for_ave[-1]:
        ind = np.where(np.logical_and(time_for_ave >= t, time_for_ave <=  t + orbit_per))
        if ind[0].size > 1:
            ave_times.append(np.mean(time_for_ave[ind]))
            ave_flux.append(np.median(flux_for_ave[ind]))
        t += orbit_per
    
    
    ave_lk = LightCurve(ave_times, ave_flux)
#     print(ave_times)
    
#     figs, ax = plt.subplots()
#     ax.plot(time_for_ave[:150], flux_for_ave[:150], 'ko')
#     ax.plot(ave_times[:10], ave_flux[:10], 'ro', ms = 5)
#     for i in range(10):
#         ax.axvline(t0+i*orbit_per)
#     plt.show()
#     return

    
    do_ave = True
#     try:
#         ave_lk = data._ave_lk_obj
#     except OS_Error:
#         do_ave = False
    
#     if ave_lk is None:
#         do_ave = False
    
    zoom_in = lk[abs(lk.time.value - np.median(lk.time.value)) <= 5]
    
    center_time = np.median(lk.time.value)
    zoom_in_2 = lk[abs(lk.time.value - np.median(lk.time.value) - 10) <= 5]
    if len(zoom_in_2.time.value) > len(zoom_in.time.value):
        zoom_in = zoom_in_2
        center_time = np.median(lk.time.value) -10 
    zoom_in_3 = lk[abs(lk.time.value - np.median(lk.time.value) + 10) <= 5]
    if len(zoom_in_3.time.value) > len(zoom_in.time.value):
        zoom_in = zoom_in_3
        center_time = np.median(lk.time.value) + 10 
    zoom_in_4 = lk[abs(lk.time.value - np.median(lk.time.value) + 20) <= 5]
    if len(zoom_in_4.time.value) > len(zoom_in.time.value):
        zoom_in = zoom_in_4
        center_time = np.median(lk.time.value) + 20
    zoom_in_5 = lk[abs(lk.time.value - np.median(lk.time.value) - 20) <= 5]
    if len(zoom_in_3.time.value) > len(zoom_in.time.value):
        zoom_in = zoom_in_3
        center_time = np.median(lk.time.value) - 20 
    
    
    c = color(name)

    axes["A"].plot(lk.time.value, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5, rasterized = True)
    try:
        axes["A"].axvline(np.min(zoom_in.time.value), color = 'silver', lw = 1, ls = '--', zorder = 5)
        axes["A"].axvline(np.max(zoom_in.time.value), color = 'silver', lw = 1, ls = '--', zorder = 5)
    except IndexError:
        pass
    axes["B"].plot(zoom_in.time.value, zoom_in.flux.value, 'ko', ms = 0.5, alpha = 0.5, rasterized = True)
    axes["A"].set_xlim(start_times[field]-5, end_times[field]+5)
    if do_ave:
        try:
            ave_zoom_in = ave_lk[abs(ave_lk.time.value - np.min(zoom_in.time.value) - 5) <= 5]
            axes["B"].plot(ave_zoom_in.time.value, ave_zoom_in.flux.value,  color = c, ls = '', marker = 'o',  ms = 1.5, alpha = 0.75, rasterized = True)
            axes["A"].plot(ave_lk.time.value, ave_lk.flux.value,  color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75, rasterized = True)
        except IndexError:
            pass
    axes["B"].set_xlim(np.min(zoom_in.time.value), np.max(zoom_in.time.value))
    pdg = data.to_periodogram(minimum_frequency = 0.1, maximum_frequency = 100)
    period = pdg.period_at_max_power.value
    period2 = 2*period
    # print(period)
    

    
    
    axes["C"].plot((lk.time.value%period)/period, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5, rasterized = True)
    axes["C"].plot((lk.time.value%period)/period + 1, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5, rasterized = True)
    axes["E"].plot((lk.time.value%period2)/period2, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5, rasterized = True)
    axes["E"].plot((lk.time.value%period2)/period2 + 1, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5, rasterized = True)
    axes["D"].axvline(1/period, color = c, lw = 2, ls = '-')
    axes["D"].axvline(1/period2, color = c, lw = 2, ls = '--')
    pdg = data.to_periodogram(minimum_frequency = 0.1, maximum_frequency = np.max([10, 2/period]), oversample_factor = 2)
    axes["D"].plot(pdg.frequency, pdg.power, 'k-', lw = 1)
    axes["D"].set_xlim(0, np.max([10, 2/period]))
    
    
    if do_ave:
        axes["C"].plot((ave_lk.time.value%period)/period, ave_lk.flux.value, color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75, rasterized = True)
        axes["C"].plot((ave_lk.time.value%period)/period + 1, ave_lk.flux.value, color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75, rasterized = True)
        axes["E"].plot((ave_lk.time.value%period2)/period2, ave_lk.flux.value,  color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75, rasterized = True)
        axes["E"].plot((ave_lk.time.value%period2)/period2 + 1, ave_lk.flux.value, color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75, rasterized = True)

    axes["B"].set_yticklabels([])
    try:
        x1 = np.round(zoom_in.time.value[0] + 2, 0)
        x3 = np.round(zoom_in.time.value[0] + 8, 0)
        x2 = np.round((x1 + x3)/2, 0)
        axes["B"].set_xticks([x1, x2, x3])
    except IndexError:
        pass
    axes["C"].set_yticklabels([])
    axes["E"].set_yticklabels([])
    axes['D'].yaxis.tick_right()
    axes['D'].yaxis.set_label_position('right')
    axes['C'].xaxis.set_label_position('top') 
    axes["A"].invert_yaxis()
    axes["B"].invert_yaxis()
    axes["C"].invert_yaxis()
    axes["E"].invert_yaxis()
    axes["A"].set_xlabel("time (days)")
    axes["B"].set_xlabel("time (days)")
    axes["C"].set_title("phase P={:3f} d".format(period))
    axes["E"].set_xlabel("phase P={:3f} d".format(period2))
    axes["D"].set_xlabel("frequency ($d^{-1}$)")
    axes["A"].set_ylabel("Magnitude")
    axes["D"].set_ylabel("Power")
    # axes["D"].set_title("f={:3f} ".format(1/period) + '${\\rm d}^{-1}$')
    axes["D"].set_title("f={:3f} ".format(1/period) + '${\\rm d}^{-1}$')
    ylim = axes["A"].get_ylim()
    axes["B"].set_ylim(ylim)
    axes["C"].set_ylim(ylim)
    axes["E"].set_ylim(ylim)
    axes["A"].set_title(data.starname + "   " + data.satellite + "    Field:" + data.field)
    # plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0, left = 0.05, right = 0.95, top = 0.85, bottom = 0.175)

    if line_top:

        axes["A"].annotate('', xy=(-1, 1.15), xycoords='axes fraction', xytext=(5, 1.175),
                    arrowprops=dict(arrowstyle="-", color='k', lw = 3))


def create_plot_array(array_of_datasets, name, file, field):
    n = len(array_of_datasets)
    
    fig = plt.figure(constrained_layout=True, figsize=(21, 29.7 / 10 * n), dpi =100)
    subfigs = fig.subfigures(n, 1, wspace = 0.5)
    for i in range(n):
        line_top = False
        if i == 0:
            line_top = False
        elif array_of_datasets[i-1].starname != array_of_datasets[i].starname:
            line_top = True
        create_plot(array_of_datasets[i]._path, subfigs[i], line_top)
    fig.patch.set_linewidth(4)
    fig.patch.set_edgecolor('black')
    plt.savefig(f"../combined_plots/{name}.pdf", facecolor = "white",edgecolor=fig.get_edgecolor())
    file.write("\\begin{figure*}\n")
    file.write("    \\centering\n")
    file.write("    \\includegraphics[width=0.85\\linewidth]{Figures/light_curves/" + name + ".pdf}\n")
    string_names = name.split(f"Field{field}")[-1]
    stars_names = string_names.split('-')
    stars_string = stars_names[0]
    try:
        for s in stars_names[1:-1]:
            stars_string = stars_string + ", " + s
    except IndexError:
        print('There is only one star')
        pass
    file.write("    \\caption{" + f"Light curves for field \\#{field} for the following stars: " + stars_string + ".}\n")
    file.write("    \\label{fig:" + name + "}\n")
    file.write("\\end{figure*}\n")
    print(f'Saved {name}.png')


In [ ]:
num = 0
# for field in [9, 10, 11, 12, 14]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for field in [9]:
        f = open(f"../combined_plots/field{field}.tex", "w")
        stars = load(field)
        array_of_datasets = []
        name = f'Field{field}'
        for star in stars:
            result_path = star.results[0]
            to_add = []
            for dataset in star.get_all_data_sets(result_path):
                if dataset.combined:
                    to_add.append(dataset)
            if len(to_add) > 11 - len(array_of_datasets):
                create_plot_array(array_of_datasets, name, f, field)
                array_of_datasets = []
                name = f'Field{field}'
            name= name + star.__str__() + '-'
            for dataset in to_add:
                array_of_datasets.append(dataset)
            if len(array_of_datasets) == 10:
                create_plot_array(array_of_datasets, name, f, field)
                array_of_datasets = []
                name = f'Field{field}'

        if array_of_datasets:
            create_plot_array(array_of_datasets, name, f, field)
        f.close()
        